In [1]:
import findspark
findspark.init()
import pyspark
from pyspark import SparkContext
import numpy as np

In [2]:
sc    = SparkContext("local[*]", "MediaVarianza_3")
datos = sc.textFile("../../../datos/data_ok.csv")
num   = datos.count() # 5000
print(num)
# datos es un RDD con 5000 strings (cada uno es una fila del csv)

5000


In [3]:
def formato_fila(fila):
    lista = []
    fila = fila.split(" ")
    for i in range(len(fila)):
        lista.append((i, float(fila[i])))
    return lista

data_flat = datos.flatMap(formato_fila) # flatmap?
sumas     = data_flat.reduceByKey(lambda x, y: x + y)
medias    = sumas.map(lambda x: (x[0], x[1]/num))

In [4]:
def formato_parc(a):
    """
    A cada elemento (col, (valor, media): Sustituyo "valor" por su valor parcial
    correspondiente para calcular la varianza.
    """
    col   = a[0]
    media = a[1][1]
    nuevo = ((a[1][0] - media)**2)/num
    return (col, (nuevo, media))

def formato_var(a, b): # (col, (parc, media)) -- sólo recibimos (parc, media)
    return (a[0] + b[0], a[1])

elem_media = data_flat.join(medias)
parciales  = elem_media.map(formato_parc)
varianzas  = parciales.reduceByKey(formato_var)

In [5]:
final = varianzas.sortByKey().collect()

for i in final:
    print("Columna %d\n\tMedia: %0.3f\n\tStdev: %0.3f"%(i[0]+1, i[1][0], i[1][1]))

Columna 1
	Media: 401818.166
	Stdev: 427.838
Columna 2
	Media: 3168373338.368
	Stdev: 8544.564
Columna 3
	Media: 2959605.643
	Stdev: 1006.771
Columna 4
	Media: 5883755024032.187
	Stdev: 1409608.915
Columna 5
	Media: 0.000
	Stdev: 1.000
Columna 6
	Media: 2.301
	Stdev: 6.654
Columna 7
	Media: 0.000
	Stdev: 1460.000
Columna 8
	Media: 313332.597
	Stdev: 762.313
Columna 9
	Media: 19828.366
	Stdev: 140.239
Columna 10
	Media: 345513253204.233
	Stdev: 506626.920
Columna 11
	Media: 6514263.902
	Stdev: 28973.706
Columna 12
	Media: 0.032
	Stdev: 0.011
Columna 13
	Media: 0.051
	Stdev: 0.946
Columna 14
	Media: 3.956
	Stdev: 7.501
Columna 15
	Media: 14631.804
	Stdev: 1411.073
Columna 16
	Media: 19686496.995
	Stdev: 3307.134
Columna 17
	Media: 103958.440
	Stdev: 198.772
Columna 18
	Media: 12358118332394.184
	Stdev: 835825.356
Columna 19
	Media: 435068936.013
	Stdev: 43811.783
Columna 20
	Media: 0.000
	Stdev: 0.000
Columna 21
	Media: 114088.276
	Stdev: 238.935
Columna 22
	Media: 151781717098.339
	Stde